In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model = "stabilityai/StableBeluga-7B"
peft_model = "Akash092003/StableBeluga-7B-finetuned-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(base_model, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", load_in_4bit=True)

model = PeftModel.from_pretrained(model, peft_model, device_map="auto", torch_dtype=torch.float16)

model = model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
import sys
sys.path.append('../../benchmarks/')

from fiqa import test_fiqa

dataset = test_fiqa(model, tokenizer)



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: @gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?
Answer: 


Total len: 150. Batchsize: 8. Total steps: 19


  0%|          | 0/19 [00:00<?, ?it/s]/home/carlos/Documents/git/llms/.env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2624: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/carlos/Documents/git/llms/.env/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
  5%|▌         | 1/19 [00:02<00:42,  2.35s/it]/home/carlos/Documents/git/llms/.env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2624: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad

Acc: 0.6466666666666666. F1 macro: 0.6466239316239316. F1 micro: 0.6466666666666666. F1 weighted (BloombergGPT): 0.6466239316239317. 


In [5]:
dataset.to_csv('../../results/financial/StableBeluga-7B-finetuned-sentiment-analysis.csv')

In [4]:
import pandas as pd

df = pd.read_csv('../../results/general/StableBeluga-7B-finetuned-sentiment-analysis.csv', index_col=0)

true, pred = df["target"], df["new_out"]


import sys
sys.path.append('../../')
from metrics import metrics

metrics(true, pred)

Precision: 0.648, Recall: 0.647, F1: 0.647, Accuracy: 0.647
